In [37]:
#Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#Model imports

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [38]:
df = pd.read_csv('data/data.csv')

In [39]:
## Let's try to predict math score
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [40]:
X = df.drop('math score',axis=1)
y = df['math score']

In [41]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [42]:
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math score, dtype: int64

In [43]:
# Create column transformer to convert categorical features to one hot encoded

num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numerical_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

preprocessor = ColumnTransformer( [
    ("OneHotEncoder",cat_transformer,cat_features), ("StandardScaler",numerical_transformer,num_features)
])

In [44]:
X = preprocessor.fit_transform(X)
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [45]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.05)

In [46]:
X_train.shape,X_test.shape

((950, 19), (50, 19))

In [49]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true,predicted)
    return mae,rmse,r2

In [53]:
models = {
    'Linear Regression' : LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'K_Nearest Neighbours': KNeighborsRegressor(),
    'Random Forest Classifier': RandomForestRegressor(),
    'XGBClassifier': XGBRegressor(),
    'Cat Boosting Regression' : CatBoostRegressor(),
    'Ada Boost Regression' : AdaBoostRegressor()
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)


    #Make Predictions

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Evaluate train and test dataset

    model_train_mae,model_train_rmse,model_train_r2 = evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for training set')
    print(f"- Root Mean Squared Error: {model_train_rmse}")
    print(f"- R2 Score : {model_train_r2}")

    print('Model performance for testing set')
    print(f"- Root Mean Squared Error: {model_test_rmse}")
    print(f"- R2 Score : {model_test_r2}")

    r2_list.append(model_test_r2)
    print(" ")

    print(40 * '-')

Linear Regression
Model performance for training set
- Root Mean Squared Error: 5.313280842654944
- R2 Score : 0.8773902177327638
Model performance for testing set
- Root Mean Squared Error: 5.501990717501144
- R2 Score : 0.8593232051685658
 
----------------------------------------
Lasso
Model performance for training set
- Root Mean Squared Error: 6.560450313458997
- R2 Score : 0.8130752197794224
Model performance for testing set
- Root Mean Squared Error: 5.732034957093292
- R2 Score : 0.8473135777835735
 
----------------------------------------
Ridge
Model performance for training set
- Root Mean Squared Error: 5.313460406275305
- R2 Score : 0.8773819303378115
Model performance for testing set
- Root Mean Squared Error: 5.495232533234615
- R2 Score : 0.859668584089789
 
----------------------------------------
K_Nearest Neighbours
Model performance for training set
- Root Mean Squared Error: 5.743565736406353
- R2 Score : 0.8567275229666844
Model performance for testing set
- Root

In [55]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['models','r2_score']).sort_values(by = ["r2_score"],ascending=False)

,models,r2_score
2,Ridge,0.859669
0,Linear Regression,0.859323
7,Ada Boost Regression,0.848785
1,Lasso,0.847314
6,Cat Boosting Regression,0.846970
4,Random Forest Classifier,0.828249
5,XGBClassifier,0.811041
3,K_Nearest Neighbours,0.752975


In [56]:
regressor = LinearRegression()
regressor.fit(X_train,y_train)
y_pred = regressor.predict(X_test)

In [59]:
pred_df = pd.DataFrame({'Actual Value' : y_test, 'Predicted Value' : y_pred, 'Difference': y_pred - y_test})
pred_df.head()

,Actual Value,Predicted Value,Difference
626,69,65.220703,-3.779297
705,62,68.824219,6.824219
795,57,62.791016,5.791016
903,93,83.869141,-9.130859
325,82,85.320312,3.320312
